In [9]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

pet_type = 'dog'
# pet_type = 'cat'

if pet_type == 'dog':
    total_pages = 10900
    base_url = 'https://search.shopping.naver.com/search/all?adQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&frm=NVSHATC&npayType=2&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&pagingIndex={}&pagingSize=40&productSet=checkout&query=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'
elif pet_type == 'cat':
    total_pages = 5900
    base_url = 'https://search.shopping.naver.com/search/all?adQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&frm=NVSHATC&npayType=2&origQuery=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&pagingIndex={}&pagingSize=40&productSet=checkout&query=%EA%B3%A0%EC%96%91%EC%9D%B4%20%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')

for idx in tqdm(range(1, len(total_pages), 100), desc='Fetching product URLs'):
    batch_ids = total_pages[idx : min(idx + 100, len(total_pages))]

    with webdriver.Chrome(options=chrome_options) as driver:
        for page in tqdm(batch_ids, desc=f'Downloading info {idx}/{len(total_pages)}', leave=False):
            try:
                url = base_url.format(page)
                driver.get(url)
                driver.implicitly_wait(2)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.implicitly_wait(2)

                links = driver.find_elements(By.CSS_SELECTOR, '.thumbnail_thumb__Bxb6Z.linkAnchor')

                for link in links:
                    try:
                        driver.get(link)
                        driver.implicitly_wait(2)
                        
                        driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
                        
                        product_name = driver.find_elements(By.CSS_SELECTOR, '._1eddO7u4UC > h3').text
                        
                        main_container = driver.find_elements(By.CSS_SELECTOR,'.se-main-container')

                        image_elements = main_container.find_elements(By.TAG_NAME, 'img')
                        image_urls = [img.get_attribute('src') for img in image_elements]
                        
                        path = f'naver/{pet_type}/{product_name}'
                            
                        if not os.path.exists(path):
                            os.makedirs(path)

                        for index, img_url in enumerate(image_urls):
                            img_data = requests.get(img_url).content
                            with open(f'{path}/image_{index + 1}.jpg', 'wb') as handler:
                                handler.write(img_data)
                    except Exception as e:
                        pass
            except Exception as e:
                pass

In [14]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

with webdriver.Chrome(options=chrome_options) as driver:
    for page in tqdm(total_pages[:1], desc='Fetching product URLs'):
        try:
            url = base_url.format(page)
            driver.get(url)
            driver.implicitly_wait(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.implicitly_wait(2)

            links = driver.find_elements(By.CSS_SELECTOR, '.thumbnail_thumb__Bxb6Z.linkAnchor')

            for link in links:
                try:
                    driver.get(link)
                    driver.implicitly_wait(2)
                    
                    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
                    
                    product_name = driver.find_elements(By.CSS_SELECTOR, '._1eddO7u4UC > h3').text
                    
                    main_container = driver.find_elements(By.CSS_SELECTOR,'.se-main-container')

                    image_elements = main_container.find_elements(By.TAG_NAME, 'img')
                    image_urls = [img.get_attribute('src') for img in image_elements]
                    
                    path = f'naver/{pet_type}/{product_name}'
                        
                    if not os.path.exists(path):
                        os.makedirs(path)

                    for index, img_url in enumerate(image_urls):
                        img_data = requests.get(img_url).content
                        with open(f'{path}/image_{index + 1}.jpg', 'wb') as handler:
                            handler.write(img_data)
                except Exception as e:
                    pass
        except Exception as e:
            pass